# EXPORTACION DE TABLAS A CSV

In [2]:
from sqlalchemy import create_engine, inspect

In [5]:
# Database conexion
host = "127.0.0.1"                 
user = "admin"                      
password = "admin"                  
database = "ctk_201801_202007"      
port = 3306                        

# Start sqlalchemy engine
path_conexion = f'mysql+pymysql://{user}:{password}@{host}:{port}/{database}'
conexion = create_engine(path_conexion)

# Listado de todas las tablas de la base de datos
inspector = inspect(conexion)
lista_tablas = inspector.get_table_names()

In [10]:
len(lista_tablas), lista_tablas[:10], lista_tablas[-10:]

(598,
 ['cms_log',
  'cms_validacion',
  'control_config',
  'control_idioma',
  'control_idioma_traduccion',
  'control_objeto',
  'control_pais',
  'control_perfil',
  'control_perfil_permiso',
  'control_perfil_usuario'],
 ['sis_transa_2034_4',
  'sis_transa_2034_5',
  'sis_transa_2034_6',
  'sis_transa_2034_7',
  'sis_transa_2034_8',
  'sis_transa_2034_9',
  'sis_transa_2050_1',
  'sis_transa_2070_0',
  'sis_transa_2070_1',
  'sis_transa_error'])

### Convert Transaction Tables

In [44]:
import numpy as np
import pandas as pd

In [58]:
a = np.arange(2018, 2021, 1, dtype=int)
b = np.arange(1, 13, 1, dtype=int)
sis_tables = [f'sis_transa_{y}_{m}' for y in a for m in b]

In [59]:
sis_tables[:5]

['sis_transa_2018_1',
 'sis_transa_2018_2',
 'sis_transa_2018_3',
 'sis_transa_2018_4',
 'sis_transa_2018_5']

In [45]:
%%time
for t in sis_tables:
    print(f'Cargando y creando tabla {t}')
    df = pd.read_sql_table(t, conexion)
    df.to_csv(f'./data_csv/{t}.csv', index=False)

Cargando y creando tabla sis_transa_2018_1
Cargando y creando tabla sis_transa_2018_2
Cargando y creando tabla sis_transa_2018_3
Cargando y creando tabla sis_transa_2018_4
Cargando y creando tabla sis_transa_2018_5
Cargando y creando tabla sis_transa_2018_6
Cargando y creando tabla sis_transa_2018_7
Cargando y creando tabla sis_transa_2018_8
Cargando y creando tabla sis_transa_2018_9
Cargando y creando tabla sis_transa_2018_10
Cargando y creando tabla sis_transa_2018_11
Cargando y creando tabla sis_transa_2018_12
Cargando y creando tabla sis_transa_2019_1
Cargando y creando tabla sis_transa_2019_2
Cargando y creando tabla sis_transa_2019_3
Cargando y creando tabla sis_transa_2019_4
Cargando y creando tabla sis_transa_2019_5
Cargando y creando tabla sis_transa_2019_6
Cargando y creando tabla sis_transa_2019_7
Cargando y creando tabla sis_transa_2019_8
Cargando y creando tabla sis_transa_2019_9
Cargando y creando tabla sis_transa_2019_10
Cargando y creando tabla sis_transa_2019_11
Cargan

In [65]:
df_transactions = pd.concat([pd.read_csv(f'./data_csv/{t}.csv') for t in sis_tables[:-5]], ignore_index=True)

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [67]:
df_transactions.to_csv(f'./data_csv/sis_transa_201801_202007.csv', index=False)

### Convert other related tables

In [52]:
fs_tables = [lista_tablas[i] for i in range(0, len(lista_tablas)) if lista_tablas[i].startswith('fs_')]

In [53]:
%%time
for t in fs_tables:
    print(f'Cargando y creando tabla {t}')
    df = pd.read_sql_table(t, conexion)
    df.to_csv(f'{t}.csv', index=False)

Cargando y creando tabla fs_aceptacion_terminos
Cargando y creando tabla fs_alarma
Cargando y creando tabla fs_asignacion_producto
Cargando y creando tabla fs_aux_departamento_cc
Cargando y creando tabla fs_bombas
Cargando y creando tabla fs_canal
Cargando y creando tabla fs_codigo_validacion
Cargando y creando tabla fs_consorcio
Cargando y creando tabla fs_consorcio_equipo
Cargando y creando tabla fs_consorcio_nombre_empresa
Cargando y creando tabla fs_contact
Cargando y creando tabla fs_departamento
Cargando y creando tabla fs_doc
Cargando y creando tabla fs_documento_terminos
Cargando y creando tabla fs_documento_terminos_idioma
Cargando y creando tabla fs_documento_terminos_perfil
Cargando y creando tabla fs_empresa
Cargando y creando tabla fs_empresa_canal
Cargando y creando tabla fs_equipo
Cargando y creando tabla fs_equipo_config
Cargando y creando tabla fs_faq
Cargando y creando tabla fs_filtro
Cargando y creando tabla fs_grupo
Cargando y creando tabla fs_grupo_departamento
Car

### ZIP CSV FILES

In [68]:
# Comprimidp en ZIP de archivos csv
import os
import zipfile
def zipdir(path, ziph):
    # ziph is zipfile handle
    for root, dirs, files in os.walk(path):
        for file in files:
            ziph.write(os.path.join(root, file))
            
zipf = zipfile.ZipFile('./data_csv.zip', 'w', zipfile.ZIP_DEFLATED)
zipdir('./data_csv/', zipf)
zipf.close()  # Importante hacerlo